In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim

import numpy as np

In [11]:
# UNIPoint model
class UNIPoint(nn.Module):
    def __init__(self, batch_size, n_inputs, n_neurons, n_parameters, n_basis_functions):
        super(UNIPoint, self).__init__()
        
        self.rnn = nn.RNNCell(n_inputs, n_neurons)
        self.hx = torch.randn(batch_size, n_neurons) # initialize hidden state 
        self.h2p = nn.Linear(n_neurons, n_parameters)
        self.parameters = torch.randn(batch_size, n_parameters) # initialize parameters for intensity functions
        self.basis_res = torch.randn(batch_size, n_parameters)
        self.Softplus = torch.nn.Softplus(beta = 1)
        self.result = torch.randn(1)

    def ReLU(self, parameters, time):
        self.output = np.maximum( (self.parameters[:,0] @ time + self.parameters[:,1]).detach().numpy(), 0 ) 
        return torch.from_numpy(self.output)
        
    def forward(self, X, time):
        output = []

        # for each time step
        for i in range(2):
            self.hx = self.rnn(X[i], self.hx)
            self.parameters = self.h2p(self.hx)
            self.basis_res = self.ReLU(self.parameters, time)
            self.result = self.Softplus(torch.sum(self.basis_res))

            output.append(self.hx)

        
        return output, self.hx, self.parameters, self.basis_res, self.result

In [19]:
# model evaluation

FIXED_BATCH_SIZE = 4 # our batch size is fixed for now
N_INPUT = 3
N_NEURONS = 1

N_PARAMETERS = 2
N_BASIS_FUNCTIONS = 8

# for now we generate random values for time in [0,T]
TIME = torch.from_numpy(np.sort(np.random.sample(FIXED_BATCH_SIZE))).type(torch.float) # TIME size is (FIXED_BATCH_SIZE, 1)

X_batch = torch.tensor([[[0,1,2], [3,4,5], 
                         [6,7,8], [9,0,1]],
                        [[9,8,7], [0,0,0], 
                         [6,5,4], [3,2,1]]], dtype = torch.float) # X0 and X1


model = UNIPoint(FIXED_BATCH_SIZE, N_INPUT, N_NEURONS, N_PARAMETERS, N_BASIS_FUNCTIONS)
print(model)
output_val, states_val, params_value, basis_value, res = model(X_batch, TIME)
print()
print(output_val) # contains all output for all timesteps
print()
print(states_val) # contains values for final state or final timestep, i.e., t=1
print()
print(params_value) # contains values for parameters for final timesteps
print()
print(basis_value) # contains values of basis functions (now it is only ReLU functions)
print()
print(res) # contains values of SoftPlus activation

UNIPoint(
  (rnn): RNNCell(3, 1)
  (h2p): Linear(in_features=1, out_features=2, bias=True)
  (Softplus): Softplus(beta=1, threshold=20)
)

[tensor([[-0.0312],
        [-0.3555],
        [-0.9933],
        [-1.0000]], grad_fn=<TanhBackward>), tensor([[-0.9760],
        [-0.2468],
        [-0.9711],
        [-0.8572]], grad_fn=<TanhBackward>)]

tensor([[-0.9760],
        [-0.2468],
        [-0.9711],
        [-0.8572]], grad_fn=<TanhBackward>)

tensor([[-0.8542,  0.1068],
        [-0.1707,  0.0128],
        [-0.8496,  0.1062],
        [-0.7429,  0.0915]], grad_fn=<AddmmBackward>)

tensor([0., 0., 0., 0.])

tensor(0.6931)
